In [1]:
!pip install yfinance
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd, numpy as np
import yfinance as yf

from copy import copy
import statistics as stats
import math
from functools import reduce

from ta.volume import MFIIndicator
from ta.volatility import AverageTrueRange
from ta.trend import STCIndicator
from ta.trend import EMAIndicator

# Import des données historiques

In [3]:
ndx = yf.Ticker("^NDX")
df_historical_data = ndx.history(interval="1d", period="max")
df_historical_data.drop(columns=["Dividends","Stock Splits"], inplace=True)
df_historical_data.reset_index(inplace=True)

In [4]:
df_historical_data.describe()

,Open,High,Low,Close,Volume
count,9451.000000,9451.000000,9451.000000,9451.000000,9.451000e+03
mean,2911.460532,2935.844467,2885.556181,2912.035138,1.604827e+09
std,3531.468398,3559.130421,3501.024969,3531.916992,1.283684e+09
min,107.160004,108.269997,106.750000,107.160004,3.174000e+07
25%,418.139999,419.620010,416.075012,418.855011,4.711300e+08
50%,1632.530029,1652.260010,1619.260010,1634.270020,1.663070e+09
75%,3692.205078,3734.349976,3644.535034,3690.589966,2.066670e+09
max,16644.769531,16764.859375,16523.830078,16573.339844,1.162119e+10


In [5]:
# Test si aucune ligne manquante
test_list = [champ == 0 for champ in df_historical_data.isnull().sum()]

# Si toutes les colonnes sont True, résultat = True
res1 = all(i for i in test_list)
res1

True

In [6]:
df_historical_data.tail(1)

,Date,Open,High,Low,Close,Volume
9450,2023-03-30 00:00:00-04:00,12935.0,12987.972656,12911.540039,12947.959961,397557327


In [7]:
df_historical_data.dtypes

Date      datetime64[ns, America/New_York]
Open                               float64
High                               float64
Low                                float64
Close                              float64
Volume                               int64
dtype: object

# Indicateurs techniques

### Alphatrend

In [8]:
# Trend indicator, équivalent de l'affichage couleur
def trend_indicator(trend):
    if trend > 0 :
        # Uptrend
        x = 1
    elif trend < 0 :
        # Downtrend
        x = -1
    else :
        # Range
        x = 0
    return x

In [13]:
# Defintion fonction
def generate_alphatrend(df_in, mfi_p, mfi_seuil, atr_l, m):
    '''Paramètres d'entrée : longueur MFI, longueur ATR, multiplier
    Retourne les colonnes Alphatrend, Alphatrend +2, Trend (position AT1 / AT2)
    :mfi_p = période MFI servant à délimiter up/down de l'alphatrend
    :mfi_seuil = période MFI pour recherche crossover, détermine uptrend ou downtrend'''

    df = df_in.copy()

    # Colonnes MFI
    s_mfi = MFIIndicator(high=df.High, low=df.Low, close=df.Close, volume=df.Volume, window=mfi_p).money_flow_index()
    df["MFI_ref"] = s_mfi

    # Colonne ATR
    s_atr = AverageTrueRange(high=df.High, low=df.Low, close=df.Close, window=atr_l).average_true_range()
    df["ATR"] = s_atr

    # Lignes UpT et DownT
    df["UpT_support"] = df["Low"] - df["ATR"] * m
    df["DownT_support"] = df["High"] + df["ATR"] * m

    # Suppression des lignes sans signal, en début de DataFrame
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    # ===============================================
    # Calcul Alphatrend, en tant que série
    
    Alphatrend = [0]

    for i in range (1, df.shape[0]):
        # Cas Uptrend
        if df.at[i,"MFI_ref"] >= mfi_seuil :
            if df.at[i,"UpT_support"] < Alphatrend[-1] :
                # Flat
                Alphatrend.append(Alphatrend[-1])
            else :
                # Trailing stop loss Up
                Alphatrend.append(df.at[i,"UpT_support"])

        # Cas Downtrend, MFI < 50
        else :
            if df.at[i,"DownT_support"] > Alphatrend[-1] :
                # Flat
                Alphatrend.append(Alphatrend[-1])
            else :
                # Trailing stop loss Down
                Alphatrend.append(df.at[i,"DownT_support"])

    # ===============================================
    # Ajout des lignes k1 et k2 en tant que colonnes
    
    if df.shape[0] == len(Alphatrend):
        df["Alphatrend_k1"] = pd.Series(Alphatrend).apply(lambda x: round(x,2))
        # Ligne k2 décalée de 2j
        Alphatrend2 = df["Alphatrend_k1"].shift(periods=2, fill_value=0)
        df["Alphatrend_k2"] = pd.Series(Alphatrend2).apply(lambda x: round(x,2))
        # Trend
        df["Trend"] = df.Alphatrend_k1 - df.Alphatrend_k2
        df["Trend"] = df["Trend"].apply(trend_indicator)
    else :
        print("Erreur lors de la génération des lignes Alphatrend")

    # ===============================================
    # Génération des signaux Achat / Vente

    # On isole tous les index non neutres, où AT1 != AT2, à la hausse (1) comme à la baisse (-1)
    s_trend = df["Trend"].loc[df["Trend"]!=0]
    s_trend_diff = s_trend - s_trend.shift(1)

    buy_signal_indexes = s_trend_diff[s_trend_diff == 2].index
    sell_signal_indexes = s_trend_diff[s_trend_diff == -2].index

    df["Signal"] = 0
    df.loc[buy_signal_indexes,"Signal"] = 1
    df.loc[sell_signal_indexes,"Signal"] = -1

    # ===============================================
    # Spécification des colonnes avec les paramètres d'entrée
    params = "MFIp = " + str(mfi_p) + ", MFItrigger = " + str(mfi_seuil) + ", ATR = " + str(atr_l) + ", m = " + str(m)
    # Sélection des colonnes suffisantes
    df = df[["Date","Alphatrend_k1","Alphatrend_k2","Trend","Signal"]]
    
    return df, params

In [14]:
df_AT, parametres_AT = generate_alphatrend(df_historical_data, mfi_p=14, mfi_seuil=50, atr_l=14, m=1.6)

In [15]:
# Servira à stocker les paramètres testés pour identifier la meilleure combinaison
print(parametres_AT)

MFIp = 14, MFItrigger = 50, ATR = 14, m = 1.6


In [16]:
df_AT.tail(1)

,Date,Alphatrend_k1,Alphatrend_k2,Trend,Signal
9437,2023-03-30 00:00:00-04:00,12530.49,12245.41,1,0


### STC & EMA

In [17]:
def generate_STC_and_EMA(df_in, stc_length, fast_length, slow_length, ema_period):
  
  df = df_in[["Date","Close"]].copy()

  s_stc = STCIndicator(close=df.Close, window_slow=slow_length, window_fast=fast_length, cycle=stc_length).stc()
  s_ema = EMAIndicator(close=df.Close, window=ema_period).ema_indicator()

  df["STC"] = round(s_stc,2)
  df["EMA"] = round(s_ema,2)

  params = "STC Length = " + str(stc_length) + ", Fast Length = " + str(fast_length) + ", Slow Length = " + str(slow_length) + ", EMA length = " + str(ema_period)

  df.drop(columns=["Close"], inplace=True)

  return df, params

In [18]:
df_STC_EMA, params_STC_EMA = generate_STC_and_EMA(df_historical_data, stc_length=80, fast_length=27, slow_length=50, ema_period=200)

In [19]:
# Servira à stocker les paramètres testés pour identifier la meilleure combinaison
print(params_STC_EMA)

STC Length = 80, Fast Length = 27, Slow Length = 50, EMA length = 200


In [20]:
df_STC_EMA.tail(1)

,Date,STC,EMA
9450,2023-03-30 00:00:00-04:00,65.51,12218.77


### Merge et export du Dataset contenant l'ensemble des indicateurs techniques

In [21]:
df_essentials = df_historical_data[["Date","Close"]].copy()
df_essentials["Close"] = df_essentials.Close.apply(lambda x: round(x,2))

In [22]:
data_frames = [df_essentials, df_AT, df_STC_EMA]
df_IT = reduce(lambda  left,right: pd.merge(left,right, on=['Date'], how='left'), data_frames)

In [23]:
df_IT = df_IT.loc[df_IT["Date"]>"1998-01-01"]

In [24]:
df_IT.head(1)

,Date,Close,Alphatrend_k1,Alphatrend_k2,Trend,Signal,STC,EMA
3099,1998-01-02 00:00:00-05:00,1008.23,995.16,995.16,0.0,0.0,5.72,998.3


In [25]:
df_IT["Trend"].value_counts()

 0.0    4115
 1.0    1601
-1.0     636
Name: Trend, dtype: int64

In [26]:
df_IT.reset_index(drop=True,inplace=True)

In [27]:
# Test si aucune ligne manquante
test_list = [champ == 0 for champ in df_historical_data.isnull().sum()]
test_list2 = [champ == 0 for champ in df_historical_data.isna().sum()]

# Si toutes les colonnes sont True, résultat = True
res1 = all(i for i in test_list)
res2 = all(i for i in test_list2)

if res1 and res2 :
  print("Ok pour Backtesting")
else :
  print("Anomalies détectées")

Ok pour Backtesting


In [28]:
df_IT.to_csv("/content/drive/MyDrive/Colab Notebooks/sources/TradingView_strategies/AT_STC_EMA_indicateurs.csv", header=True, index=False)

# Backtesting

## Détection des entrées